# FargoCPT Bootstrap Notebook

## What is this?
This notebook can be used to setup a FargoCPT simulation in an empty directory.

## Why is this useful?
This is useful when you want to run a simulation on a cluster which you can access using Jupyter notebooks and want to store a full copy of the code alongside your simulation outputs.

## Contents

It will do the following things:
- clone the repository from github
- compile the code
- give you a command to run the simulation
- print the available data
- plot the surface density

First we create a new directory and change to it.

In [1]:
example_name = "300_bootstrap"
example_dir = f"example_dirs/{example_name}"
import os
if not os.path.basename(os.getcwd()) == example_name:
    !mkdir -p $example_dir
    os.chdir(example_dir)
repo_root = os.path.abspath(os.path.join(os.getcwd(), "../../../"))
print(f"Current working directory: {os.getcwd()}")
print(f"Repository root directory: {repo_root}")

Current working directory: /home/rometsch/repo/fargocpt/examples/example_dirs/300_bootstrap
Repository root directory: /home/rometsch/repo/fargocpt


## Downloading the code

We will clone only the last commit of the code which is enough to run the simulation and faster to download.

In [2]:
!git clone --depth 1 https://github.com/rometsch/fargocpt code

Cloning into 'code'...


remote: Enumerating objects: 749, done.
remote: Counting objects: 100% (749/749), done.
remote: Compressing objects: 100% (625/625), done.
remote: Total 749 (delta 155), reused 459 (delta 88), pack-reused 0
Receiving objects: 100% (749/749), 4.95 MiB | 4.37 MiB/s, done.
Resolving deltas: 100% (155/155), done.


## Building the code

Make sure the code is built by running make again.

In [3]:
%%timeit -n1 -r1
from sys import platform
if platform in ["linux", "darwin"]:
    !make -j 4 -C code/src > make.log
else:
    raise RuntimeError(f"Seems like you are not running MacOS or Linux but {platform}. This is unsupported. You are on your own, good luck!")

26.7 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


## Preparing a setup file

We'll take the example setup file from the examples directory and modify it in python.
If you want to create setup files for a parameter study, just copy the code and make your own setup creator script.

In [4]:
configfile = "setup.yml"
!cp code/examples/config.yml $configfile

We'll use the `ruamel.yaml` package to read and write the setup file. This can be set up to preserve comments which is very useful if you want to trace your decisions later on.

In [5]:
try:
    import ruamel.yaml
except ImportError:
    raise ImportError("Please install ruamel.yaml with `python3 -m pip install ruamel.yaml`")
yaml = ruamel.yaml.YAML()
with open(configfile, "r") as infile:
    config = yaml.load(infile)

In [6]:
config["nbody"][1]["accretion efficiency"] = "2"
config["MonitorTimestep"] = 0.314 # monitor scalar files around every half orbit
config["Nmonitor"] = 20 # write a snapshot every orbit
config["Nsnapshots"] = 30 # wirte 10 snapshots
# use very low resolution by setting it to 2 cell per scaleheight, cps
del config["Nrad"]
del config["Naz"]
config["cps"] = 2

In [7]:
with open(configfile, "w") as outfile:
    yaml.dump(config, outfile)

## Running the code

We can start fargo using the python interface, but this runs slower when started from within a Jupyter Notebook compared to being executed from without.
Even calling a python script using the shell magic "!" does not speed it up.
Calling a python script that does the same job from the command line does not have this issue.

If anyone knows why this is, please let me know!

For a production run, even for further testing, please open a terminal and run the output of the following cell:

In [8]:
cwd = os.getcwd()
cmd = f"cd {cwd} && code/run_fargo -np 1 -nt 4 auto {configfile}"
print(cmd)

cd /home/rometsch/repo/fargocpt/examples/example_dirs/300_bootstrap && code/run_fargo -np 1 -nt 4 auto setup.yml


If you plan to run on a cluster, put something similar into a run script to be queued in the queueing system.

Consider generating your queuing script here so you can just copy this notebook and setup a new simulation.

For the sake of this notebook, we just run the simluation here.

We'll use the python module from within the directory.

In [9]:
import sys
sys.path = ["code/python"] + sys.path
from fargocpt import run
run(["start", configfile], np=2, nt=1, exe="code/bin/fargocpt_exe", detach=False)

Running command: mpirun -np 2 --report-pid /tmp/tmpcqh1chwn -x OMP_NUM_THREADS=1 code/bin/fargocpt_exe start setup.yml
fargo process pid 1403025

[0] MPI rank #  0 runs as process 1403029
[1] MPI rank #  1 runs as process 1403030
[0] MPI rank #  0 OpenMP thread #  0 of  1 on cpt-kamino
[1] MPI rank #  1 OpenMP thread #  0 of  1 on cpt-kamino
[0] fargo: This file was compiled on Nov 14 2023, 15:52:04.
[0] fargo: This version of FARGO used _GNU_SOURCE
[0] fargo: This version of FARGO used NDEBUG. So no assertion checks!
[0] Using parameter file setup.yml
[0] Computing disk quantities within 5.00000e+00 L0 from coordinate center
[0] BC: Inner composite = reflecting
[0] BC: Outer composite = reflecting
[0] BC: Sigma inner = zerogradient
[0] BC: Sigma outer = zerogradient
[0] BC: Energy inner = zerogradient
[0] BC: Energy outer = zerogradient
[0] BC: Vrad inner = reflecting
[0] BC: Vrad outer = reflecting
[0] BC: Vaz inner = keplerian
[0] BC: Vaz outer = keplerian
[0] DampingTimeFactor: 1.0

0

## Data

Let's check whether there is data.

In [10]:
from fargocpt import Loader

In [11]:
l = Loader("output/out")
l

   Loader
| output_dir: output/out
| snapshots: 0 ... 30
| special_snapshots: ['reference']
| snapshot_time: 0.0 5.02257e+06 s ... 188.4 5.02257e+06 s
| monitor_number: 0 ... 600
| units: Units
| target_units = None
| gas: Hydro
| nbody: Nbody
| params: Params
| particles = None